Name    : Risky Armansyah Email : rarmnsyah787@gmail.com

In [1]:
!pip install split-folders 

In [2]:
import tensorflow as tf
import zipfile,os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import splitfolders
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.image as mpimg

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

C:\Users\LENOVO\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip\
  -O /tmp/rockpaperscissors.zip

local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
base_dir = '/tmp/rockpaperscissors'

splitfolders.ratio(os.path.join(base_dir, 'rps-cv-images'), output = base_dir, ratio = (.6, .4), seed = 1337)

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In [ ]:
train_gen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      horizontal_flip=True,
      shear_range = 0.2,
      fill_mode = 'nearest'
)

test_gen = ImageDataGenerator(
      rescale = 1./255
)

In [ ]:
train_generator = train_gen.flow_from_directory(
    train_dir, 
    target_size = (100,150), 
    class_mode = 'categorical' 
)

validation_generator = test_gen.flow_from_directory(
    val_dir, 
    target_size=(100, 150), 
    class_mode='categorical'
)

In [ ]:
# callbacks
accuracy_threshold = 96e-2
class my_callbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if logs.get('accuracy') >= accuracy_threshold:
            print('\nFor Epoch', epoch, '\nAkurasi mencapai = %2.2f%%' %(logs['accuracy']*100), 'proses pelatihan akan dihentikan.')
            self.model.stop_training = True

In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (100,150,3)),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(256, (3,3), activation = 'relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation = 'relu'),
#     tf.keras.layers.Dense(3, activation = 'softmax')
# ])

model = tf.keras.models.Sequential()
model.add(Conv2D(32, (3,3), activation = 'relu', input_shape = (100,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation = 'relu', input_shape = (100,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(128, (3,3), activation = 'relu', input_shape = (100,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256, (3,3), activation = 'relu', input_shape = (100,150,3)))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(3, activation = 'softmax'))

model.summary()

In [ ]:
model.compile(
    optimizer = tf.optimizers.Adam(),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 25,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 5,
    verbose = 2,
    callbacks = [my_callbacks()]
)

In [ ]:
%matplotlib inline

uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(100,150))
 
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
 
  classes = model.predict(images, batch_size=10).argmax()
  print(fn)
  print(classes, model.predict(images, batch_size=10))
  if classes==0:
   print('paper')
  elif classes == 1:
    print('rock')
  else:
   print('scissor')